Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [22]:
reset -fs

In [23]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [24]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [25]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [26]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [39]:
batch_size = 128
graph = tf.Graph()

with graph.as_default():
    
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_valid_labels = tf.constant(valid_labels)
    tf_test_dataset = tf.constant(test_dataset)
    tf_test_labels = tf.constant(test_labels)
    
    weights = tf.Variable(tf.truncated_normal([ image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    # batch_size x 764 @ 764 x 10, then plus 1 x 10 on each row
    logits = tf.matmul(tf_train_dataset,weights) + biases
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels)
                         + 0.01* tf.nn.l2_loss(weights)
                         # + 0.01* tf.nn.l2_loss(biases)
                         )
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset,weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset,weights) + biases)

In [40]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size),:]
        batch_labels = train_labels[offset:(offset + batch_size),:]
        feed_dict = {
            tf_train_dataset: batch_data,
            tf_train_labels: batch_labels
        }
        
        _, l, predictions = session.run([optimizer,loss, train_prediction],feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
        
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 45.499081
Minibatch accuracy: 10.2%
Validation accuracy: 15.2%
Minibatch loss at step 500: 0.809219
Minibatch accuracy: 84.4%
Validation accuracy: 81.0%
Minibatch loss at step 1000: 0.800516
Minibatch accuracy: 81.2%
Validation accuracy: 81.8%
Minibatch loss at step 1500: 0.782825
Minibatch accuracy: 81.2%
Validation accuracy: 82.1%
Minibatch loss at step 2000: 0.656436
Minibatch accuracy: 82.8%
Validation accuracy: 82.4%
Minibatch loss at step 2500: 0.754628
Minibatch accuracy: 80.5%
Validation accuracy: 82.3%
Minibatch loss at step 3000: 0.797186
Minibatch accuracy: 83.6%
Validation accuracy: 81.3%
Test accuracy: 87.9%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [32]:
num_steps = 3001

train_dataset_small = train_dataset[:batch_size*5]
train_labels_small = train_labels[:batch_size*5]

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels_small.shape[0] - batch_size)
        batch_data = train_dataset_small[offset:(offset + batch_size),:]
        batch_labels = train_labels_small[offset:(offset + batch_size),:]
        feed_dict = {
            tf_train_dataset: batch_data,
            tf_train_labels: batch_labels
        }
        
        _, l, predictions = session.run([optimizer,loss, train_prediction],feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
        
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 46.212212
Minibatch accuracy: 14.8%
Validation accuracy: 14.3%
Minibatch loss at step 500: 0.546461
Minibatch accuracy: 100.0%
Validation accuracy: 77.1%
Minibatch loss at step 1000: 0.365506
Minibatch accuracy: 100.0%
Validation accuracy: 77.8%
Minibatch loss at step 1500: 0.364359
Minibatch accuracy: 100.0%
Validation accuracy: 77.7%
Minibatch loss at step 2000: 0.364351
Minibatch accuracy: 100.0%
Validation accuracy: 77.7%
Minibatch loss at step 2500: 0.364351
Minibatch accuracy: 100.0%
Validation accuracy: 77.7%
Minibatch loss at step 3000: 0.364351
Minibatch accuracy: 100.0%
Validation accuracy: 77.7%
Test accuracy: 84.1%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [45]:
batch_size = 128
graph = tf.Graph()

with graph.as_default():
    
    keep_prob = tf.placeholder(tf.float32)
    tf_train_dataset_original = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
    tf_train_dataset = tf.nn.dropout(tf_train_dataset_original,0.5)
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_valid_labels = tf.constant(valid_labels)
    tf_test_dataset = tf.constant(test_dataset)
    tf_test_labels = tf.constant(test_labels)
    
    weights = tf.Variable(tf.truncated_normal([ image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    # batch_size x 764 @ 764 x 10, then plus 1 x 10 on each row
    def compute_logits(x, weights, biases):
        
        logits = tf.matmul(x,weights) + biases
       
        return logits
    
    logits = compute_logits(tf_train_dataset, weights, biases)
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels)
                         + 0.01* tf.nn.l2_loss(weights)
                         + 0.01* tf.nn.l2_loss(biases)
                         )
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(compute_logits(tf_valid_dataset,weights,biases))
    test_prediction = tf.nn.softmax(compute_logits(tf_test_dataset,weights,biases))

In [46]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size),:]
        batch_labels = train_labels[offset:(offset + batch_size),:]
        feed_dict = {
            tf_train_dataset_original: batch_data,
            tf_train_labels: batch_labels
        }
        
        _, l, predictions = session.run([optimizer,loss, train_prediction],feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
        
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 59.675903
Minibatch accuracy: 7.0%
Validation accuracy: 11.4%
Minibatch loss at step 500: 0.831341
Minibatch accuracy: 82.0%
Validation accuracy: 80.7%
Minibatch loss at step 1000: 0.924292
Minibatch accuracy: 77.3%
Validation accuracy: 81.0%
Minibatch loss at step 1500: 0.888206
Minibatch accuracy: 79.7%
Validation accuracy: 81.0%
Minibatch loss at step 2000: 0.807989
Minibatch accuracy: 78.9%
Validation accuracy: 81.6%
Minibatch loss at step 2500: 0.880462
Minibatch accuracy: 75.0%
Validation accuracy: 81.8%
Minibatch loss at step 3000: 0.937357
Minibatch accuracy: 78.1%
Validation accuracy: 80.8%
Test accuracy: 87.3%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [58]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

    # Network Parameters
    n_input = 784 # 28x28 image size
    n_hidden_1 = 1024
    n_classes = 10
    
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    keep_prob = tf.placeholder(tf.float32)
    
    x = tf.placeholder(tf.float32, shape=(None, n_input))
    x_dropout = tf.nn.dropout(x,keep_prob)
    y = tf.placeholder(tf.float32, shape=(None, n_classes))
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables. Store layers weight & bias

    weights = {
        'h1': tf.Variable(tf.truncated_normal([n_input, n_hidden_1])),
        'out': tf.Variable(tf.truncated_normal([n_hidden_1, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.truncated_normal([n_hidden_1])),
        'out': tf.Variable(tf.truncated_normal([n_classes]))
    }

    # Training computation with 1 hidden layer
    def multiple_perception(x, weights, biases):
        layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
        layer_1 = tf.nn.relu(layer_1)

        outlayer = tf.matmul(layer_1,weights['out'])+ biases['out']
        return outlayer

    outlayer = multiple_perception(x_dropout, weights, biases)

    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=outlayer)
        + 0.01* tf.nn.l2_loss(weights['h1'])
        + 0.01* tf.nn.l2_loss(weights['out'])
        + 0.01* tf.nn.l2_loss(biases['b1'])
        + 0.01* tf.nn.l2_loss(biases['out'])
    )

    # Optimizer.
    
    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, 128, 0.96)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    # optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(outlayer)
    # Validation
    valid_prediction = tf.nn.softmax(multiple_perception(tf_valid_dataset, weights, biases))
    # Test Prediction
    test_prediction = tf.nn.softmax(multiple_perception(tf_test_dataset, weights, biases))


In [59]:
num_steps = 3001 #3001

# def accuracy(predictions, labels):
#     return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {x : batch_data, y : batch_labels, keep_prob: 0.5}
        valid_dict = {x : valid_dataset, y : valid_labels, keep_prob: 1.0}
        _, l, p = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        #print(predictions)
        #print(batch_labels)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(p,batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(feed_dict=valid_dict),valid_labels))    
            
    test_dict = {x: test_dataset, y: test_labels, keep_prob: 1.0}
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(feed_dict=test_dict),test_labels))
                  

Initialized
Minibatch loss at step 0: 3677.411865
Minibatch accuracy: 9.4%
Validation accuracy: 30.3%
Minibatch loss at step 500: 30.157518
Minibatch accuracy: 84.4%
Validation accuracy: 81.8%
Minibatch loss at step 1000: 1.391874
Minibatch accuracy: 82.8%
Validation accuracy: 83.8%
Minibatch loss at step 1500: 0.812751
Minibatch accuracy: 82.0%
Validation accuracy: 84.0%
Minibatch loss at step 2000: 0.676165
Minibatch accuracy: 86.7%
Validation accuracy: 84.2%
Minibatch loss at step 2500: 0.786470
Minibatch accuracy: 82.0%
Validation accuracy: 84.3%
Minibatch loss at step 3000: 0.777123
Minibatch accuracy: 82.0%
Validation accuracy: 84.8%
Test accuracy: 90.5%
